In [1]:
import pandas as pd
import os
from datetime import datetime


### Set data sources and parameters

In [2]:
apc_dir = r'C:\data\OneDrive\WSP O365\chattanooga-modeling - RTP TDM Update\Data\CARTA_PassengerData\CARTA APC DATA 2019'

gtfs_dir = r'C:\data\OneDrive\WSP O365\chattanooga-modeling - RTP TDM Update\Data\TransitUpdate2\GTFS'
# gtfs_dir = r'C:\apps\client_chattanooga_rtp_update\Data\GTFS\carta_2019_08_15'

datatypes = {'STOP_ID': str, 'ROUTE_NUMBER':str, 'stop_id': str}

In [15]:
route_xref = pd.DataFrame(data=[
    ['1.40',   'Route #1 am HC:1',  '1',   'ALTON PARK'],
    ['10.10',  'Route #10A',        '10A', 'AVON'],
    ['10.60',  'Route #10C',        '10C', 'CAMPBELL'],
    ['10.90',  'Route #10G 58:10G', '10G', 'GLENWOOD'],
    ['13.00',  'Route #13',         '13',  'ROSSVILLE'],
    ['14.00',  'Route #14',         '14',  'MOCS EXPRESS'],
    ['15.00',  'Route #15',         '15',  'ST. ELMO'],
    ['16.40',  'Route #16 In:16',   '16',  'NORTHGATE OB'],
    ['19.00',  'Route #19',         '19',  'CROMWELL ROAD'],
    ['2.00',   'Route #2',          '2',   'NORTH CHATT'],
    ['21.00',  'Route #21',         '21',  'GOLDEN GATEWAY'],
    ['28.00',  'Route #28',         '28',  'AMNICOLA HWY CHATT STATE'],
    ['33.00',  'Route #DTS:33',     '33',  'DOWNTOWN SHUTTLE'],
    ['34.00',  'Route #34',         '34',  'NORTH SHORE SHUTTLE'],
    ['4.00',   'Route #4',          '4',   'EASTGATE/HAMILTON PL'],
    ['7.00',   'Route #7',          '7',   'CHATTANOOGA HOUSING AUTHORITY'],
    ['780.00', '3:3',               '3',   'ENTERPRISE SOUTH'],
    ['8.00',   'Route #8',          '8',   'EASTDALE'],
    ['9.00',   'Route #9',          '9',   'EAST LAKE'],
    ],
    columns=['ROUTE_NUMBER', 'ROUTE_NAME', 'route_id', 'route_short_name']
)

In [4]:
### AUGUST
apc_file = 'AUG2019_RIDECHECK_DATA.TXT'

dates_pick = [
             1,
    6,   7,  8,
    13, 14, 15,
    20, 21, 22,
    27, 28, 29
]

dates_pick = ['8/{}/2019 0:00:00'.format(d) for d in dates_pick]

### Load APC and GTFS Data

In [29]:
apc = pd.read_csv(os.path.join(apc_dir, apc_file), dtype=datatypes)
apc = pd.merge(apc, route_xref[['ROUTE_NUMBER', 'route_id', 'route_short_name']], on='ROUTE_NUMBER', how='left')
# list(apc)
# apc = apc[['ROUTE_NUMBER', 'ROUTE_NAME','STOP_ID', 'MAIN_CROSS_STREET']].copy()

apc_stops = apc.groupby(['STOP_ID', 'MAIN_CROSS_STREET'])['STOP_ID'].count().reset_index(name='count')
apc_stops['MAIN_CROSS_STREET'] = apc_stops['MAIN_CROSS_STREET'].str.replace('/', ' + ')

gtfs = pd.read_csv(os.path.join(gtfs_dir, 'stops.txt'), dtype=datatypes)
gtfs = gtfs[['stop_id', 'stop_name']]

C:\Anaconda3\envs\chattanooga\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (14,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
# apc[['ROUTE_NUMBER', 'ROUTE_NAME']].groupby(['ROUTE_NUMBER', 'ROUTE_NAME'])['ROUTE_NUMBER'].count().reset_index(name='_count')

##### Verify APC to GTFS Stop_ID using intersection data


In [32]:
stops_check = pd.merge(apc_stops, gtfs, left_on='STOP_ID', right_on='stop_id', how='left')

matches = stops_check[stops_check['MAIN_CROSS_STREET']==stops_check['stop_name']]
not_matches = stops_check[stops_check['MAIN_CROSS_STREET']!=stops_check['stop_name']]

print('APC Stations: {:,}'.format(len(apc_stops)))
print('GTFS Stations: {:,}'.format(len(gtfs)))
print('APC - GTFS Stations difference: {:,}'.format(len(apc_stops) - len(gtfs)))

print('Station EXACT Matches: {:,} ratio: {:.2f}'.format(len(matches), len(gtfs) / len(apc_stops)))

not_matches.loc['MAIN_CROSS_STREET'] = not_matches['MAIN_CROSS_STREET'].str.replace(' ', '')
not_matches.loc['stop_name'] = not_matches['stop_name'].str.replace(' ', '')
close_matches = not_matches[not_matches['MAIN_CROSS_STREET']==not_matches['stop_name']]
print('Station CLOSE Matches: {:,}'.format(len(close_matches)))

print('Station NOT Matches: {:,}'.format(len(not_matches)-len(close_matches)))

null_stop_id = not_matches[not_matches['stop_name'].isna()]
print('APC Stations ID NOT in GTFS: {:,}'.format(len(null_stop_id)))

APC Stations: 1,240
GTFS Stations: 1,181
APC - GTFS Stations difference: 59
Station EXACT Matches: 1,143 ratio: 0.95
Station CLOSE Matches: 0
Station NOT Matches: 99
APC Stations ID NOT in GTFS: 89


C:\Anaconda3\envs\chattanooga\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
# matches.sample(10)

### Route level average data boardings by TOD
Weekdays:
    Tue
    Wed
    Thurs
    
August 2019 (13 obs)

Populate table 'Transit Route Boardings':  
https://wsponline.sharepoint.com/:w:/r/sites/US-chattanoogamodeling/RTP%20TDM%20Update/Deliverables/Task%203_Model%20Validation/Chattanooga%20TPO%20Model%20Validation%20Report.docx

In [34]:
## CHECKS on TOD
# apc[['SURVEY_DATE', 'TRIP_START_TIME', 'TIME_PERIOD', 'TIME_SCHEDULED', 'TIME_ACTUAL_ARRIVE', 'TIME_ACTUAL_DEPART',]].sample(5)
# apc['TIME_PERIOD'].unique()

# test[(test['TIME_PERIOD']=='AM Early')&(test['ROUTE_NUMBER']==7)].groupby('SURVEY_DATE').count()
# test[(test['TIME_PERIOD']=='PM Late')&(test['ROUTE_NUMBER']==2)].groupby('SURVEY_DATE').count()

# apc.groupby(['TIME_PERIOD']).agg({'TRIP_START_TIME':['min', 'max']})

##### Update Time Of Day periods

In [35]:
tod_windows = {
    'AM':{'from': '6', 'to': '9'},
    'PM':{'from': '15', 'to': '18'}
}

apc['trip_start_astime'] = [datetime.strptime(t, '%m/%d/%Y %H:%M:%S').time() for t in apc['TRIP_START_TIME']] 

apc['TOD'] = 'OP'  ## All other times are set to Off-Peak

for t in tod_windows.keys():
    apc.loc[apc['trip_start_astime'].between(datetime.strptime((tod_windows[t]['from']), '%H').time(),
                                             datetime.strptime((tod_windows[t]['to']), '%H').time()
                                    ), 'TOD'] = t

##### Time of Day average

In [36]:
apc_pick = apc[apc['SURVEY_DATE'].isin(dates_pick)]
apc_pick = apc_pick[['SERIAL_NUMBER', 'SURVEY_DATE', 'ROUTE_NUMBER', 'route_id', 'route_short_name', 'TOD']]

weekdays = apc_pick.groupby(['ROUTE_NUMBER', 'route_id', 'route_short_name', 'SURVEY_DATE', 'TOD']).count()\
                   .groupby(['ROUTE_NUMBER', 'route_id', 'route_short_name', 'TOD']).mean().unstack()


round(weekdays).astype(int).to_clipboard()
round(weekdays).astype(int)

SERIAL_NUMBER            
TOD                                                            AM    OP    PM
ROUTE_NUMBER route_id route_short_name                                       
1.40         1        ALTON PARK                              518  1568   479
10.10        10A      AVON                                    318   818   283
10.60        10C      CAMPBELL                                207   255   169
10.90        10G      GLENWOOD                                424  1065   436
13.00        13       ROSSVILLE                               411   832   596
15.00        15       ST. ELMO                                263   265   292
16.40        16       NORTHGATE OB                            531  1346   539
19.00        19       CROMWELL ROAD                           269   395   234
2.00         2        NORTH CHATT                              73   170   140
21.00        21       GOLDEN GATEWAY                          112   477   220
28.00        28       AMNICOLA HWY CHATT STATE                128   247   141
33.00        33       DOWNTOWN SHUTTLE                        417  1761   574
34.00        34       NORTH SHORE SHUTTLE                      95   291   117
4.00         4        EASTGATE/HAMILTON PL                   1614  3998  1588
7.00         7        CHATTANOOGA HOUSING AUTHORITY            44   264    55
780.00       3        ENTERPRISE SOUTH                         98    87   103
8.00         8        EASTDALE                                337   876   395
9.00         9        EAST LAKE                               572  1618   574

##### Time of Day average, including NaN as Zero

In [ ]:
apc_pick = apc[['SERIAL_NUMBER', 'SURVEY_DATE', 'ROUTE_NUMBER', 'TOD']]

routes = pd.DataFrame(index=set(apc['ROUTE_NUMBER'])).sort_index()
routes.index.name = 'ROUTE_NUMBER'

# tod_cols = list(set(apc['TIME_PERIOD']))
# tod_cols= ['AM Early', 'AM Peak', 'Midday', 'PM Peak', 'PM Late', 'Other']

tod_cols = ['OP']
tod_cols.extend(tod_windows.keys())

apc_dict = {}
for day_pick in dates_pick:
    apc_day_pick = apc_pick[apc_pick['SURVEY_DATE']==day_pick]
    apc_dict[day_pick] = apc_day_pick.groupby(['ROUTE_NUMBER', 'SURVEY_DATE', 'TOD']).count()\
                                     .groupby(['ROUTE_NUMBER', 'TOD']).mean().unstack().fillna(0)
    
    apc_dict[day_pick].columns = apc_dict[day_pick].columns.get_level_values(1)

    # Fill Routes
    apc_dict[day_pick] = routes.join(apc_dict[day_pick], how='left').fillna(0)
    
    # Fill Columns
    for col in tod_cols:
        if col not in apc_dict[day_pick]:
            apc_dict[day_pick][col]=0

    apc_dict[day_pick] = apc_dict[day_pick][tod_cols]
    
weekdays = sum(apc_dict.values())/len(apc_dict.values())
round(weekdays).astype(int)

##### CHECK 10/31 Holiday